In [1]:
import pickle
import numpy as np
import xgboost as xgb
import glob
import pandas as pd

fl = sorted(glob.glob('../features/*.pkl'))
for l in fl:
    print("'{}',".format(l))

'../features/cnn_1_aug1_feat.pkl',
'../features/cnn_1_aug_rescale_preprocess_feat.pkl',
'../features/cnn_1_aug_skimage_preprocess_feat.pkl',
'../features/cnn_1_feat.pkl',
'../features/cnn_2_aug1_feat.pkl',
'../features/cnn_2_aug_rescale_preprocess_feat.pkl',
'../features/cnn_2_aug_skimage_preprocess_feat.pkl',
'../features/cnn_2_feat.pkl',
'../features/cnn_3_aug1_feat.pkl',
'../features/cnn_3_aug_rescale_preprocess_feat.pkl',
'../features/cnn_3_aug_skimage_preprocess_feat.pkl',
'../features/cnn_4_aug1_feat.pkl',
'../features/cnn_4_aug1_feat_add_early.pkl',
'../features/cnn_4_aug_rescale_preprocess_feat.pkl',
'../features/cnn_4_aug_skimage_preprocess_feat.pkl',
'../features/cnn_5_aug1_feat.pkl',
'../features/cnn_5_aug_rescale_preprocess_feat.pkl',
'../features/cnn_5_aug_skimage_preprocess_feat.pkl',
'../features/incept_1_feat.pkl',
'../features/incept_aug1_feat.pkl',
'../features/incept_aug2_feat.pkl',
'../features/incept_aug3_feat.pkl',
'../features/incept_aug4_feat.pkl',
'../features/

In [2]:
choose_fl = [
 '../features/cnn_1_aug1_feat.pkl',
 '../features/cnn_1_aug_rescale_preprocess_feat.pkl',
 '../features/cnn_1_aug_skimage_preprocess_feat.pkl',
 '../features/cnn_1_feat.pkl',
 '../features/cnn_2_aug1_feat.pkl',
# '../features/cnn_2_aug_rescale_preprocess_feat.pkl',
# '../features/cnn_2_aug_skimage_preprocess_feat.pkl',
# '../features/cnn_2_feat.pkl',
# '../features/cnn_3_aug1_feat.pkl',
# '../features/cnn_3_aug_rescale_preprocess_feat.pkl',
 '../features/cnn_3_aug_skimage_preprocess_feat.pkl',
 '../features/cnn_4_aug1_feat.pkl',
# '../features/cnn_4_aug1_feat_add_early.pkl',
# '../features/cnn_4_aug_rescale_preprocess_feat.pkl',
 '../features/cnn_4_aug_skimage_preprocess_feat.pkl',
# '../features/cnn_5_aug1_feat.pkl',
# '../features/cnn_5_aug_rescale_preprocess_feat.pkl',
 '../features/cnn_5_aug_skimage_preprocess_feat.pkl',
 '../features/incept_1_feat.pkl',
 '../features/incept_aug1_feat.pkl',
# '../features/incept_aug2_feat.pkl',
# '../features/incept_aug3_feat.pkl',
# '../features/incept_aug4_feat.pkl',
 '../features/other_feat.pkl',
 '../features/resnet_1_feat.pkl',
 '../features/resnet_aug1_feat.pkl',
# '../features/resnet_aug2_feat.pkl',
# '../features/resnet_aug3_feat.pkl',
 '../features/resnet_aug4_feat.pkl',
# '../features/vgg16_1_feat.pkl',
 '../features/vgg_aug1_feat.pkl',
 '../features/xception_aug1_feat.pkl',
]

def load_x_y():
    train_x,test_x=[],[]
    for f in choose_fl:
        print(f)
        with open(f,'rb') as fin:
            a,b = pickle.load(fin)
            train_x.append(a)
            test_x.append(b)
    train_x = np.hstack(train_x)
    test_x = np.hstack(test_x)
    train_df = pd.read_json('../input/train.json')
    y = train_df.is_iceberg.values
    return train_x,test_x,y

train_x,test_x,train_y = load_x_y()
print(train_x.shape,test_x.shape,train_y.shape)

../features/cnn_1_aug1_feat.pkl
../features/cnn_1_aug_rescale_preprocess_feat.pkl
../features/cnn_1_aug_skimage_preprocess_feat.pkl
../features/cnn_1_feat.pkl
../features/cnn_2_aug1_feat.pkl
../features/cnn_3_aug_skimage_preprocess_feat.pkl
../features/cnn_4_aug1_feat.pkl
../features/cnn_4_aug_skimage_preprocess_feat.pkl
../features/cnn_5_aug_skimage_preprocess_feat.pkl
../features/incept_1_feat.pkl
../features/incept_aug1_feat.pkl
../features/other_feat.pkl
../features/resnet_1_feat.pkl
../features/resnet_aug1_feat.pkl
../features/resnet_aug4_feat.pkl
../features/vgg_aug1_feat.pkl
../features/xception_aug1_feat.pkl
(1604, 27) (8424, 27) (1604,)


In [5]:
from sklearn.linear_model import LogisticRegression
lr_m = LogisticRegression()
lr_m.fit(train_x,train_y)
train_pred = lr_m.predict_proba(train_x)
from sklearn.metrics import log_loss
print(log_loss(train_y,train_pred))
print(log_loss(train_y,np.clip(train_pred,0.001,0.999)))
print(log_loss(train_y,np.clip(train_pred,0.005,0.995)))

0.150869880584
0.150912477172
0.151255727198


In [4]:
test_df=pd.read_json('../input/test.json')
submission = pd.DataFrame()
submission['id']=test_df['id']
test_pred = lr_m.predict_proba(test_x)[:,1]
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/lr_ens_some.csv', index=False)

         id  is_iceberg
0  5941774d    0.018219
1  4023181e    0.933798
2  b20200e4    0.020080
3  e7f018bb    0.990468
4  4371c8c3    0.899914
